In [1]:
import requests
import json
from arcgis.apps.storymap import StoryMap, Themes, Image, Video, Audio, Embed, Map, Button, Text, Gallery, Timeline, Sidecar, Code, Table, TextStyles, Collection, CollectionNavigation
import arcgis
from arcgis.gis import GIS, Item
import os
from PIL import Image as PILImage
import tempfile
from io import BytesIO
from copy import deepcopy

In [2]:
# Print the version of the arcgis module
print(f"Running ArcGIS API for Python version: {arcgis.__version__}")
agoNotebook = False
# Define the GIS
if agoNotebook == False:
    try:
        import keyring
        service_name = "system" # Use the default local credential store
        success = False # Set initial state

        # Ask for the username
        while success == False:
            username_for_keyring = input("Enter your ArcGIS Online username:") # If you are using VS Code, the text input dialog box appears at the top of the window
            # Get the credential object
            credential = keyring.get_credential(service_name, username_for_keyring)
            # Check if the username is in the credential store
            if credential is None:
                print(f"'{username_for_keyring}' is not in the local system's credential store. Try another username.")
            # Retrieve the password, login and set the GIS portal
            else:
                password_from_keyring = keyring.get_password("system", username_for_keyring)
                portal_url = 'https://www.arcgis.com'  
                gis = GIS(portal_url, username=username_for_keyring, password=password_from_keyring)
                success = True
                # Print a success message with username and user's organization role
                print(f"Successfully logged in as: {gis.properties.user.username} (role: {gis.properties.user.role} userType: {gis.properties.user.userLicenseTypeId})")
    except ImportError:
        print("The 'keyring' module is not installed. Please install it using 'pip install keyring'.")
        print("Before re-running this cell, open a command line window on your machine and run the command:")
        print("# python -m keyring set system <your_ago_username>")
        print("If using Windows Powershell, use:")
        print("# ./python -m keyring set system <your_ago_username>")
        print("You will be prompted to enter your password")
        print("When you hit Enter/Return the password will be saved to your local credential store.")
else:
    gis = GIS("home")

Running ArcGIS API for Python version: 2.4.1.3
Successfully logged in as: dasbury_storymaps (role: org_admin userType: GISProfessionalAdvUT)


In [ ]:
def fetch_classic_storymap_data(classic_storymap_id, gis):
    classic_item = Item(gis=gis, itemid=classic_storymap_id)
    classic_data = Item.get_data(classic_item)
    if classic_data == {}:
        raise ValueError("ERROR: StoryMap to be converted must be hosted on ArcGIS Online.")
    elif isinstance(classic_data, dict):
        classic_item_data = classic_data
    else:
        classic_item_data = json.loads(classic_data)
    return classic_item, classic_item_data

def extract_story_settings(classic_item_data):
    settings = classic_item_data["values"]["settings"]
    title = classic_item_data["values"].get("title", "Untitled StoryMap")
    subtitle = classic_item_data["values"].get("subtitle", "")
    story_type = settings["layout"]["id"]
    panel_position = settings["layoutOptions"]["panel"]["position"]
    theme = settings["theme"]
    entries = classic_item_data["values"]["story"]["entries"]
    return title, subtitle, story_type, panel_position, theme, entries

def build_webmap_from_json(gis, media):
    """
    Build a minimal webmap JSON for the print service from a storymap entry's media property,
    using the basemap from the referenced webmap item if available.
    """
    # Default basemap (fallback)
    basemap_url = "https://services.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer"

    basemap_layer = {
        "url": basemap_url,
        "visibility": True,
        "opacity": 1,
        "id": "World_Topo_Map"
    }
    baseMap = {
        "baseMapLayers": [basemap_layer],
        "title": "World Topographic Map"
    }

    # Try to get basemap from the referenced webmap item
    if "webmap" in media and "id" in media["webmap"]:
        try:
            webmap_item = gis.content.get(media["webmap"]["id"])
            wm_data = webmap_item.get_data()
            if "baseMap" in wm_data:
                baseMap = wm_data["baseMap"]
        except Exception as e:
            print(f"Could not fetch basemap from webmap item: {e}")

    webmap_json = {
        "baseMap": baseMap,
        "operationalLayers": [],
        "spatialReference": {"wkid": 102100}
    }

    # Extent
    extent = None
    if "webmap" in media and "extent" in media["webmap"]:
        extent = media["webmap"]["extent"]
        webmap_json["mapOptions"] = {"extent": extent}
        webmap_json["extent"] = extent

    # Layers (same as before)
    if "webmap" in media and "layers" in media["webmap"]:
        for lyr in media["webmap"]["layers"]:
            layer_url = None
            if "id" in media["webmap"]:
                try:
                    webmap_item = gis.content.get(media["webmap"]["id"])
                    wm_data = webmap_item.get_data()
                    for op_lyr in wm_data.get("operationalLayers", []):
                        if op_lyr.get("id") == lyr["id"]:
                            layer_url = op_lyr.get("url")
                            break
                except Exception as e:
                    print(f"Could not fetch webmap item for layer URL lookup: {e}")

            if not layer_url and "url" in lyr:
                layer_url = lyr["url"]

            if layer_url:
                op_layer = {
                    "url": layer_url,
                    "visibility": lyr.get("visibility", True),
                    "id": lyr["id"]
                }
                webmap_json["operationalLayers"].append(op_layer)

    # Add export options for print service
    webmap_json["exportOptions"] = {"outputSize": [800, 600], "dpi": 96}

    return webmap_json

def process_entry(gis, entry, default_thumbnail_path):
    entry_title = entry.get("title")
    media_info = entry.get("media", {})
    media_type = media_info.get("type")
    main_stage_content = None
    thumbnail_path = None
    thumbnail = None
    invalid_webmap = False

    if media_type == "webmap":
        webmap_id = media_info.get('webmap', {}).get('id')
        webmap_from_json = build_webmap_from_json(gis, media_info)
        if webmap_from_json:
            try:
                thumbnail_path, thumbnail = create_webmap_thumbnail(webmap_json=webmap_from_json, default_thumbnail_path=default_thumbnail_path)
            except Exception as e:
                print(f"Error processing webmap {entry_title} ({webmap_id}): {e}")
                invalid_webmap = True            
        if webmap_id and not invalid_webmap:
            try:
                main_stage_content = Map(webmap_id)
            except Exception as e:
                print(f"Error creating Map object for {entry_title} ({webmap_id}): {e}")
                invalid_webmap = True
    elif media_type == "webpage":
        webpage_url = media_info.get("webpage", {}).get("url")
        if webpage_url:
            main_stage_content = Embed(webpage_url)
    elif media_type == "image":
        image_url = media_info.get("image", {}).get("url")
        if image_url:
            main_stage_content = Image(image_url)
            thumbnail_path, thumbnail = create_image_thumbnail(image_url=image_url, default_thumbnail_path=default_thumbnail_path)
            if not os.path.isfile(thumbnail_path):
                thumbnail_path = default_thumbnail_path

    if not thumbnail_path or not os.path.isfile(thumbnail_path):
        thumbnail_path = default_thumbnail_path

    return entry_title, main_stage_content, thumbnail_path, thumbnail, invalid_webmap

def create_image_thumbnail(image_url, default_thumbnail_path):
    try:
        response = requests.get(image_url)
        img = PILImage.open(BytesIO(response.content))
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        img.thumbnail((800, 600))
        img.save(temp_file.name)
        return temp_file.name, img
    except Exception:
        print("Thumbnail download failed; using default.")
        img = PILImage.open(BytesIO(requests.get(default_thumbnail_path).content))
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
        img.thumbnail((800, 600))
        img.save(temp_file.name)
        return default_thumbnail_path, img

def remove_failed_service(webmap_json, failed_url):
    # Remove from operationalLayers
    if 'operationalLayers' in webmap_json:
        webmap_json['operationalLayers'] = [
            lyr for lyr in webmap_json['operationalLayers']
            if not lyr.get('url', '').startswith(failed_url)
        ]
    # Remove from baseMapLayers
    if 'baseMap' in webmap_json and 'baseMapLayers' in webmap_json['baseMap']:
        webmap_json['baseMap']['baseMapLayers'] = [
            lyr for lyr in webmap_json['baseMap']['baseMapLayers']
            if not lyr.get('url', '').startswith(failed_url)
        ]
    return webmap_json

# For downloads, use in-memory BytesIO where possible
def create_webmap_thumbnail(webmap_json, default_thumbnail_path):
    url = "https://utility.arcgisonline.com/arcgis/rest/services/Utilities/PrintingTools/GPServer/Export%20Web%20Map%20Task/execute"
    #webmap_json = webmap_item.get_data()
    webmap_json = webmap_json if isinstance(webmap_json, dict) else json.loads(webmap_json)
    webmap_json_copy = deepcopy(webmap_json)
    tried_urls = set()
    max_attempts = 10  # Prevent infinite loops

    # List to capture all print service responses
    print_service_responses = []

    # Ensure exportOptions is set
    if 'exportOptions' not in webmap_json_copy:
        webmap_json_copy['exportOptions'] = {
            "outputSize": [800, 600],
            "dpi": 96
        }
    # Ensure mapOptions/extent is set
    if 'mapOptions' not in webmap_json_copy:
        webmap_json_copy['mapOptions'] = {}
    if 'extent' not in webmap_json_copy['mapOptions']:
        webmap_json_copy['mapOptions']['extent'] = webmap_json.get('mapOptions', {}).get('extent', webmap_json.get('initialState', {}).get('viewpoint', {}).get('targetGeometry'))

    for attempt in range(max_attempts):
        params = {
            "f": "json",
            "Web_Map_as_JSON": json.dumps(webmap_json_copy),
            "Format": "PNG32",
            "Layout_Template": "MAP_ONLY"
        }
        response = requests.post(url, data=params)
        result = response.json()

        # Capture the response for troubleshooting
        print_service_responses.append({
            "attempt": attempt + 1,
            "params": params,
            "status_code": response.status_code,
            "result": result
        })

        if 'results' in result:
            image_url = result['results'][0]['value']['url']
            img_response = requests.get(image_url)
            if img_response.status_code == 200:
                temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
                temp_file.write(img_response.content)
                temp_file.close()
                img = PILImage.open(temp_file.name)
                return temp_file.name, img
            else:
                break  # No valid image, break and use default

        elif 'error' in result and 'details' in result['error']:
            # Try to extract the failed service URL
            failed_layer_detail = result['error']['details'][0]
            if ' at ' in failed_layer_detail:
                failed_service_url = failed_layer_detail.split(' at ')[-1]
                if failed_service_url in tried_urls:
                    break  # Prevent infinite loop if same URL keeps failing
                tried_urls.add(failed_service_url)
                webmap_json_copy = remove_failed_service(webmap_json_copy, failed_service_url)
                continue  # Try again with the updated JSON
            else:
                break  # Can't parse the failed URL, break and use default
        else:
            break  # No results and no error details, break and use default

    # If we reach here, fallback to default
    print("Thumbnail download failed; using default.")
    default_path, img = create_image_thumbnail(image_url=default_thumbnail_path, default_thumbnail_path=default_thumbnail_path)
    return default_path, img

In [15]:
item_id = "597d573e58514bdbbeb53ba2179d2359"
default_thumbnail_path = "https://cdn-a.arcgis.com/cdn/1BE082D/js/arcgis-app-components/arcgis-app/assets/arcgis-item-thumbnail/storymap.png"
classic_item, classic_item_data = fetch_classic_storymap_data(classic_storymap_id=item_id, gis=gis)
title, subtitle, story_type, panel_position, theme, entries = extract_story_settings(classic_item_data)
for entry in entries:
    thumbnail_paths = []
    thumbnails = []
    entry_title, main_stage_content, thumbnail_path, thumbnail, invalid_webmap, print_service_responses = process_entry(gis, entry, default_thumbnail_path=default_thumbnail_path)
    thumbnail_paths.append(thumbnail_path)
    thumbnails.append(thumbnail)

In [14]:
if path and os.path.isfile(path):
    try:
        # If you have a reference to the PIL Image object, close it first
        if path in thumbnail_paths and thumbnails[thumbnail_paths.index(path)]:
            thumbnails[thumbnail_paths.index(path)].close()
        os.remove(path)
        print(f"Deleted: {path}")
    except Exception as e:
        print(f"Could not delete {path}: {e}")

Deleted: C:\Users\davi6569\AppData\Local\Temp\2\tmp6ryvizrc.png
